# 动态提示词
<img src="./assets/LC_DynamicPrompts.png" width="500">

## 设置

加载并/或检查所需的环境变量

In [12]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")


DASHSCOPE_API_KEY=****0fbe
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****5ced
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [13]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [14]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [15]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [16]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

你是一名谨慎的 SQLite 分析员。规则：

按步骤思考。
需要数据时，用单条 SELECT 查询调用 execute_sql 工具。
只读；不能使用 INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE。
限制为 5 行，除非用户明确要求更多。
{table_limits}
如果工具返回 “Error:”，请修改 SQL 后再试。
优先使用显式列名，避免 SELECT *。

## 构建动态提示词
利用运行时上下文和中间件来生成动态提示词。

In [17]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
        #仅限访问这些表：Album、Artist、Genre、Playlist、PlaylistTrack、Track。
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

在 `create_agent` 中加入中间件。

In [18]:
from langchain_qwq import ChatQwen
import os
llm=ChatQwen(
    model="qwen3-max",
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

In [19]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [ ]:
question = "Frank Harris 最昂贵的一次购买是什么？"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Frank Harris 最昂贵的一次购买是什么？
================================== Ai Message ==================================

要确定 Frank Harris 最昂贵的一次购买，我们需要查看与他相关的购买记录。然而，在当前可访问的表（Album、Artist、Genre、Playlist、PlaylistTrack、Track）中，并没有包含客户信息或购买记录的表（如 Customer 或 Invoice）。因此，无法直接查询 Frank Harris 的购买信息。

如果您有其他相关表的信息或更多上下文，请提供进一步的细节！


In [23]:
question = "Frank Harris 最昂贵的一次购买是什么？"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Frank Harris 最昂贵的一次购买是什么？
================================== Ai Message ==================================

我需要找出 Frank Harris 最昂贵的一次购买。首先，我需要确认数据库中是否有包含客户姓名和购买信息的表。
Tool Calls:
  execute_sql (call_9d0bb26b2b5b4d5ca2a24584)
 Call ID: call_9d0bb26b2b5b4d5ca2a24584
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================

我看到有 `Customer` 表和 `Invoice` 表，可能还有 `InvoiceLine` 表包含购买详情。首先，我会查看 `Customer` 表的结构，以确认如何找到 Frank Harris。
Tool Calls:
  execute_sql (call_7bfbcb4928be435ab7bda4c0)
 Call ID: call_7bfbcb4928be435ab7bda4c0
  Args:
    query: PRAGMA table_info(